In [2]:
import pandas as pd
import numpy as np
import Levenshtein

train_data = pd.read_csv("train.csv")
train_update = pd.read_csv("train_updates_20220929.csv")

First we need to delete some error data, and replace some data according to train_updates_20220929.csv

In [3]:
nrow_update = train_update.shape[0]
nrow_train = train_data.shape[0]
update_proseq = train_update.iloc[:,1]

# By judging whether the protein sequence is NaN, find the rows to be deleted
row_ind_delete = list(train_update.iloc[list(update_proseq!=update_proseq),0])

# Find the rows that are to be modified
row_ind_replace = list(train_update.iloc[list(update_proseq==update_proseq),0])

# Now construct a list of rows to substitude the target rows
substitution = train_update.iloc[list(update_proseq==update_proseq),:]
substitution = [list(substitution.iloc[i,:]) for i in range(substitution.shape[0])]

# Now find the rows that should be kept, instead be being deleted
row_ind_remain = list(range(nrow_train))
row_ind_remain = list(set(row_ind_remain)-set(row_ind_delete))

# Finally, acquire the modified train data set
modified_train_data = train_data
for row in substitution:
    row_ind = row[0]
    modified_train_data.iloc[row_ind,:] = row
modified_train_data = modified_train_data.iloc[row_ind_remain,:]


In [4]:
len_list = modified_train_data.protein_sequence.str.len()
modified_train_data["seq_length"] = len_list
modified_train_data.sort_values(by="seq_length" , inplace=True, ascending=True) 
len_list = sorted(len_list)

C:\Users\lsk\AppData\Local\Temp\ipykernel_2424\3759851295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_train_data["seq_length"] = len_list
C:\Users\lsk\AppData\Local\Temp\ipykernel_2424\3759851295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_train_data.sort_values(by="seq_length" , inplace=True, ascending=True)


In [5]:
# This is the rough grouping, which group all the samples only by their similarity 
# in the protein sequences
modified_train_data["new_id"] = list(range(len(len_list)))

group_extend_scale = 1              # Here we resgirct that the sequences in the same group can only have one mutation
group_lower_bound = 5
already_grouped = []
groups = []
groups_ind = []
groups_len = []

xx = np.array(list(range(len(len_list))))

for ind_x in range(len(len_list)):    
    if ind_x in already_grouped:
        continue
    length = len_list[ind_x]
    seq = modified_train_data.iloc[ind_x, 1]
    neighbor_bar = group_extend_scale
    selected_rows = list((xx > ind_x) &
                        (modified_train_data.seq_length <= neighbor_bar+length))
    if len(selected_rows) == 0:
        continue
    extended_neighbor = list(modified_train_data.iloc[selected_rows,1])
    group_member_ind = [ind_x]
    maybe_grouped = []
    for neighbor_ind in range(len(extended_neighbor)):
        neighbor = extended_neighbor[neighbor_ind]
        difference = Levenshtein.distance(neighbor, seq)
        
        # Here, if a sequence is simliar to seq, put it
        # into the group, and label it "grouped". 
        if difference <= 2:             # Resgirct that the sequences in the same group can only have one mutation
            group_member_ind.append(ind_x+neighbor_ind+1)
            maybe_grouped.append(ind_x+neighbor_ind+1)

    # If the group size reaches the bar, record the indices of samples in 
    # the group and formally label the grouped samples
    if len(group_member_ind) > group_lower_bound:
        already_grouped += maybe_grouped
        groups_ind.append(group_member_ind)
        groups_len.append(len(group_member_ind))

C:\Users\lsk\AppData\Local\Temp\ipykernel_2424\2511352343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_train_data["new_id"] = list(range(len(len_list)))


In [6]:
# Observe the distribution of grouping
print(len(groups_ind))
print(sum(groups_len))
print(groups_len)

for ind in range(len(groups_ind)):
    sample = modified_train_data.iloc[groups_ind[ind], 2:]
    lengths = len(list(set(sample.seq_length)))
    if lengths >= 2:
        print(ind, lengths)

94
4347
[52, 21, 53, 20, 45, 6, 55, 13, 127, 27, 62, 78, 32, 10, 12, 78, 6, 8, 59, 8, 11, 8, 78, 68, 8, 54, 194, 7, 6, 124, 49, 180, 34, 10, 41, 28, 60, 708, 12, 144, 9, 17, 12, 9, 19, 6, 6, 16, 19, 273, 16, 26, 151, 6, 53, 21, 13, 10, 12, 28, 17, 7, 10, 25, 10, 8, 36, 10, 54, 10, 7, 12, 51, 7, 6, 7, 31, 19, 130, 61, 211, 30, 6, 24, 84, 10, 72, 12, 6, 8, 13, 8, 10, 17]
31 2
35 2
43 2
45 2
54 2


In [7]:
# Output the dataframe
category = [-1] * len(len_list)
for category_ind in range(len(groups_ind)):
    group = groups_ind[category_ind]
    for ind in group:
        category[ind] = category_ind 
modified_train_data["group"] = category
tt = modified_train_data[modified_train_data["group_id"]>-1]
print(tt.shape)
tt.to_csv("grouped_sample_no_pH_one_muta.csv",index=False)

(4347, 9)


C:\Users\lsk\AppData\Local\Temp\ipykernel_39192\2820161087.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_train_data["group"] = category


In [ ]:
# Re-group the samples within the groups we already got
# by putting samples of the same pH values and sources
# together 
cleared_group = []
for ind in range(len(groups_ind)):
    group = groups_ind[ind]
    samples = modified_train_data.iloc[group, :]
    back_ground_info = list(set(list(samples.pH)))
    for group_criteria in back_ground_info:
        small_group = samples.iloc[ np.array(samples.pH==group_criteria), 6]
        small_group = list(small_group)
        if len(small_group) >= 5:
            cleared_group.append(small_group)


In [ ]:
# This block tries to see wether there are groups containing sequences of 
# different lengths
print(len(cleared_group))
for ind in range(len(cleared_group)):
    sample = modified_train_data.iloc[cleared_group[ind], 2:]
    lengths = len(list(set(sample.seq_length)))
    if lengths >= 2:
        print(ind, lengths)

In [ ]:
# Output the dataframe
category = [0] * len(len_list)
for category_ind in range(len(cleared_group)):
    group = cleared_group[category_ind]
    for ind in group:
        category[ind] = category_ind + 1
modified_train_data["group_id"] = category
modified_train_data.to_csv("grouped_sample_pH.csv",index=False)